In [1]:
from jppype import Mosaic, vscode_theme

from fundus_vessels_toolkit.pipelines import FundusVesselSegToGraph
from fundus_vessels_toolkit.segment import clear_gpu_cache, segment_vessels
from fundus_vessels_toolkit.segment_to_graph import populate_geometry
from fundus_vessels_toolkit.utils.data_io import load_image

vscode_theme()

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

#### Load image

In [2]:
# Path to the raw fundus image
RAW_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/test/downsampled-1024/1-images/30_A.jpg"

# Load the raw image
raw = load_image(RAW_PATH)

# Segment the vessels
vessels = segment_vessels(raw)

# Clear the GPU cache
clear_gpu_cache()

#### Parse the vascular graph AND its geometry

In [3]:
max_vessel_diameter = raw.shape[1] // 50
seg2graph = FundusVesselSegToGraph(max_vessel_diameter, parse_geometry=True)
graph = seg2graph(vessels)

#### Display the graph

In [4]:
mosaic = Mosaic(3, raw, cell_height=800, cols_titles=["Topology", "Cleaned Skeleton", "BSplines"])
mosaic.add_label(vessels, "vessel", opacity=0.3)

mosaic[0]["vgraph"] = graph.jppype_layer(node_labels=True)
mosaic[1]["vgraph"] = graph.jppype_layer(edge_map=True, edge_labels=True, boundaries=True)
mosaic[2]["vgraph"] = graph.jppype_layer(bspline=True, edge_labels=True)
mosaic[2]["bsplines_nodes"] = graph.geometric_data().jppype_bspline_tangents(scaling=0.4)

mosaic.show()

GridBox(children=(HTML(value='<h3 style="text-align: center;">Topology</h3>'), HTML(value='<h3 style="text-ali…

### Access geometrical data

The geometrical representations of vascular graphs are stored in ``VGeometricData`` objects. Usually ``VGraph``  are linked to a single ``VGeometricData`` object, but actually it may be linked to more than one when several geometrical versions of the same graph are available (e.g. different image of the same patient registered together). In any case, the geometrical data can be accessed through the ``VGraph`` objects by calling the ``geometric_data()`` method (with an optional id).

``VGeometricData`` object are only containers and only provide method to access the geometrical data, they also ensure that the data stay coherent when the graph is modified (e.g. when calling``graph.delete_nodes()``, ``VGraph.reindex_branches()``, ``VGraph.merge_nodes()``, ``VGraph.split_branches()``, etc). 

However they don't provide tools to generate the geometrical data. This is done by the ``fundus_vessels_toolkit.segment_to_graph.geometry_parsing.populate_geometry()`` method or by enabling the ``parse_geometry`` option in the ``FundusVesselSegToGraph`` pipelines.

In [5]:
geo_data = graph.geometric_data()
list(geo_data.branch_data().keys())

['TIPS_TANGENTS',
 'TIPS_CALIBRE',
 'TIPS_BOUNDARIES',
 'TANGENTS',
 'BSPLINE',
 'CALIBRE',
 'BOUNDARIES',
 'CURVATURES']

#### Extract bifurcation infos

In [6]:
from fundus_vessels_toolkit import VBranchGeoData
from fundus_vessels_toolkit.utils.graph.measures import extract_bifurcations_parameters

# Access the calibre and tangent data for each branch
# Those metrics were previously computed when calling ``seg2graph(vessels)`` with parse_geometry=True
geo_data = graph.geometric_data()
calibres = geo_data.branch_data(VBranchGeoData.Fields.CALIBRES)
tangents = geo_data.branch_data(VBranchGeoData.Fields.TANGENTS)
extract_bifurcations_parameters(calibres, tangents, graph.branch_list, False)

,b0,b1,b2,θ1,θ2,d0,d1,d2
nodeID,,,,,,,,
0,72,79,78,-0.681651,26.530780,15.100000,13.408051,5.666995
1,36,35,34,0.342868,51.062572,12.019228,11.992634,3.543999
3,13,60,66,12.533369,58.056009,5.384840,3.821475,4.760817
6,19,17,18,17.083047,34.590419,11.931830,11.633673,4.441327
7,62,61,10,35.989537,45.204278,8.287713,5.600193,5.036010
9,42,41,40,26.655125,47.457889,6.186934,4.798383,4.846637
11,19,21,20,9.889036,117.547506,10.977593,10.936964,1.887691
13,45,46,38,14.791903,74.952374,10.406836,8.289933,5.627383
17,78,82,81,42.096520,44.400070,9.782988,9.359948,4.740928
